In [1]:
! pip install ipympl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.1 MB/s eta 0:00:00


In [66]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

from public_tests_a1 import *

tf.keras.backend.set_floatx('float64')
from assigment_utils import *

tf.autograph.set_verbosity(0)

## **2-Evaluating a Learning Algorithm (Polynomial Regression)**

In [4]:
# Generate some data
X,y,x_ideal,y_ideal = gen_data(18, 2, 0.7)
print("X.shape", X.shape, "y.shape", y.shape)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=1)
print("X_train.shape", X_train.shape, "y_train.shape", y_train.shape)
print("X_test.shape", X_test.shape, "y_test.shape", y_test.shape)

X.shape (18,) y.shape (18,)
X_train.shape (12,) y_train.shape (12,)
X_test.shape (6,) y_test.shape (6,)


## **Exercise 1**
Below, create a function to evaluate the error on a data set for a linear regression model.

In [5]:
# UNQ_C1
# GRADED CELL: eval_mse
def eval_mse(y, yhat):
    """
    Calculate the mean squared error on a data set.
    Args:
      y    : (ndarray  Shape (m,) or (m,1))  target value of each example
      yhat : (ndarray  Shape (m,) or (m,1))  predicted value of each example
    Returns:
      err: (scalar)
    """
    m = len(y)
    err = 0.0
    for i in range(m):
    ### START CODE HERE ###
      err = err + (yhat[i] - y[i])**2
    err = err / (2*m)
    return err
    ### END CODE HERE ###

    return(err)

In [7]:
# Test
y_hat = np.array([2.4, 4.2])
y_tmp = np.array([2.3, 4.1])
test_eval_mse = eval_mse(y_tmp, y_hat)
print(test_eval_mse)

0.0050000000000000305


In [20]:
# Compare J(train) and J(test)
degree = 10

X_train_trans = PolynomialFeatures(degree).fit_transform(X_train.reshape(-1,1))
X_test_trans = PolynomialFeatures(degree).fit_transform(X_test.reshape(-1,1))

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

linear_model = LinearRegression()
linear_model.fit(X_train_trans, y_train)

yhat_train = linear_model.predict(X_train_trans)
yhat_test = linear_model.predict(X_test_trans)

err_train = mean_squared_error(y_train,yhat_train)
err_test = mean_squared_error(y_test,yhat_test)

print(f"training err {err_train:0.2f}, test err {err_test:0.2f}")

training err 1574.97, test err 46010.11


In [43]:
import inspect
args = inspect.getargspec(lin_model)
print(args)

args = inspect.getargspec(tune_m)
print(args)

ArgSpec(args=['self', 'degree', 'regularization', 'lambda_'], varargs=None, keywords=None, defaults=(False, 0))
ArgSpec(args=[], varargs=None, keywords=None, defaults=None)


<ipython-input-43-8743e927c694>:2: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args = inspect.getargspec(lin_model)
<ipython-input-43-8743e927c694>:5: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args = inspect.getargspec(tune_m)


In [21]:
# create a model in sklearn, train on training data
degree = 10
lmodel = lin_model(degree)
lmodel.fit(X_train, y_train)

# predict on training data, find training error
yhat = lmodel.predict(X_train)
err_train = lmodel.mse(y_train, yhat)

# predict on test data, find error
yhat = lmodel.predict(X_test)
err_test = lmodel.mse(y_test, yhat)

print(f"training err {err_train:0.2f}, test err {err_test:0.2f}")

training err 58.01, test err 171215.01


In [22]:
# Generate  data
X,y, x_ideal,y_ideal = gen_data(40, 5, 0.7)
print("X.shape", X.shape, "y.shape", y.shape)

# Splt data into 60% Training data; 20% Cross Validation data; 20% Test data
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5,random_state=1) # cv for cross validation
print("X_train.shape", X_train.shape, "y_train.shape", y_train.shape)
print("X_cv.shape", X_cv.shape, "y_cv.shape", y_cv.shape)
print("X_test.shape", X_test.shape, "y_test.shape", y_test.shape)

X.shape (40,) y.shape (40,)
X_train.shape (24,) y_train.shape (24,)
X_cv.shape (8,) y_cv.shape (8,)
X_test.shape (8,) y_test.shape (8,)


In [35]:
# Train model repeatedly to find the optimized degree
degree_max = 9

for i in range(degree_max):
    lmodel = lin_model(i+1)
    lmodel.fit(X_train, y_train)

    yhat_train = lmodel.predict(X_train)
    yhat_cv = lmodel.predict(X_cv)

    err_train = lmodel.mse(y_train, yhat_train)
    err_cv = lmodel.mse(y_cv, yhat_cv)

    print(f"degree {i+1:0.2f}, training err {err_train:0.2f}, cv err {err_cv:0.2f}")

degree 1.00, training err 33075.06, cv err 57076.44
degree 2.00, training err 10284.87, cv err 39591.79
degree 3.00, training err 8414.67, cv err 42671.91
degree 4.00, training err 5572.78, cv err 60706.12
degree 5.00, training err 4983.98, cv err 63952.44
degree 6.00, training err 4735.37, cv err 72688.75
degree 7.00, training err 4008.19, cv err 115892.50
degree 8.00, training err 3820.88, cv err 129498.68
degree 9.00, training err 3780.27, cv err 140864.18


In [34]:
# Multinomial to fix high bias
err_train1 = np.zeros(degree_max)
err_cv1 = np.zeros(degree_max)

for i in range(degree_max):
    lmodel = lin_model(i+1)
    lmodel.fit(X_train, y_train)

    yhat_train = lmodel.predict(X_train)
    err_train1[i] = lmodel.mse(y_train, yhat_train)
    yhat_cv = lmodel.predict(X_cv)
    err_cv1[i] = lmodel.mse(y_cv, yhat_cv)

print(err_cv1)
optimize = np.argmin(err_cv1) + 1
print(f"degree {optimize:0.2f}")

[ 57076.44  39591.79  42671.91  60706.12  63952.44  72688.75 115892.5
 129498.68 140864.18]
degree 2.00


In [39]:
# Regularization to fix high variance
lambda_range = np.array([0.0, 1e-6, 1e-5, 1e-4,1e-3,1e-2, 1e-1,1,10,100])
err_cv2 = np.zeros(len(lambda_range))
degree = 10

for i in range(len(lambda_range)):
    lmodel = lin_model(degree, regularization=True,lambda_=lambda_range[i])
    lmodel.fit(X_train, y_train)

    yhat_cv = lmodel.predict(X_cv)
    err_cv2[i] = lmodel.mse(y_cv, yhat_cv)

print(err_cv2)
optimize = np.argmin(err_cv2)
print(f"lambda {lambda_range[optimize]:0.2f}")

[332318.16 115064.09  98995.61  76072.54  64245.44  58922.77  56996.52
  54961.41  68104.59 119626.42]
lambda 1.00


In [44]:
# Collecting more data to fix high variance
X_train, y_train, X_cv, y_cv, x, y_pred, err_train, err_cv, m_range,degree = tune_m() # tuning numbers of examples

## **3 - Evaluating a Learning Algorithm (Neural Network)**

In [45]:
# Generate and split data set
X, y, centers, classes, std = gen_blobs() # Classification Dataset

# Splitting Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5,random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.2,random_state=1)

In [81]:
# Below, compose a three-layer model:

# Dense layer with 120 units, relu activation
# Dense layer with 40 units, relu activation
# Dense layer with 6 units and a linear activation (not softmax)
# Compile using
# loss with SparseCategoricalCrossentropy, remember to use from_logits=True
# Adam optimizer with learning rate of 0.01.
tf.random.set_seed(1234)
model = Sequential(
    [
        Dense(120,activation=relu,name='l1'),
        Dense(40,activation=relu,name='l2'),
        Dense(6,activation=linear,name='l3')
    ],name = "Complex"
)

model.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.01)
)

model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
13/13 [==============================] - 1s 3ms/step - loss: 1.1077
Epoch 2/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4235
Epoch 3/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.3166
Epoch 4/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2993
Epoch 5/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2914
Epoch 6/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.3143
Epoch 7/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2519
Epoch 8/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.2356
Epoch 9/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2395
Epoch 10/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2025
Epoch 11/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2155
Epoch 12/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.2054
Epoch 13/1000
13/13 [====

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               360       
                                                                 
 dense_1 (Dense)             (None, 40)                4840      
                                                                 
 dense_2 (Dense)             (None, 6)                 246       
                                                                 
Total params: 5446 (42.55 KB)
Trainable params: 5446 (42.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [85]:
pred = model.predict(X_cv)
pred = tf.nn.softmax(pred).numpy()
pred_class = np.argmax(pred, axis=1)
print(pred_class)

10/10 [==============================] - 0s 2ms/step
[2 0 0 5 4 3 2 0 4 0 3 2 2 3 3 3 0 5 1 1 3 2 3 4 5 4 0 1 4 4 2 1 1 5 1 5 0
 2 4 0 2 1 4 2 5 4 4 3 0 2 5 1 3 0 2 4 0 4 1 4 3 4 1 2 2 0 4 4 3 5 2 0 3 5
 0 1 0 2 1 0 0 4 5 3 4 0 4 1 5 3 2 5 1 2 4 1 1 3 1 2 0 1 5 4 2 3 4 5 3 3 5
 1 2 3 3 5 0 5 5 0 4 3 1 4 0 4 2 3 4 1 3 2 0 4 3 2 4 2 1 2 3 4 0 3 2 5 4 0
 2 2 4 5 2 5 5 5 0 1 0 3 4 0 3 1 4 2 0 0 2 0 4 0 3 1 2 4 1 1 4 0 2 3 3 0 0
 2 4 4 1 5 5 3 5 3 5 1 3 0 4 2 0 5 5 1 3 5 2 4 0 3 0 4 1 1 2 5 0 1 3 5 3 3
 1 4 5 1 5 3 2 3 4 4 2 3 5 4 4 0 1 0 2 5 4 5 2 3 0 3 5 0 1 0 0 0 0 2 1 3 3
 2 1 3 2 0 0 1 3 4 1 4 3 4 2 1 5 2 5 3 3 2 4 2 2 3 2 5 4 2 1 0 5 2 5 1 3 5
 2 0 3 2 1 4 1 5 5 5 3 3 0 0 4 2 2 3 2 4 5 5 1 4]


In [56]:
# Compute erro
def cat_err(y,yhat):
  err = 0
  for i in range(len(y)):
    if y[i] != yhat[i]:
      err = err + 1
    else:
      err = err + 0
  err = err / len(y)
  return err

In [86]:
pred_cv = model.predict(X_cv)
pred_cv = tf.nn.softmax(pred_cv).numpy()
yhat_cv = np.argmax(pred_cv, axis=1)

pred_train = model.predict(X_train)
pred_train = tf.nn.softmax(pred_train).numpy()
yhat_train = np.argmax(pred_train, axis=1)

train_err = cat_err(y_train,yhat_train)
cv_err = cat_err(y_cv,yhat_cv)

print(f"training err {train_err:0.3f}, cv err {cv_err:0.3f}")

13/13 [==============================] - 0s 2ms/step
training err 0.013, cv err 0.103


In [82]:
# lower training error, higher cross validation error
# higher variance
# try simple model

# Dense layer with 6 units, relu activation
# Dense layer with 6 units and a linear activation. Compile using
# loss with SparseCategoricalCrossentropy, remember to use from_logits=True
# Adam optimizer with learning rate of 0.01.
tf.random.set_seed(1234)
model_simple = Sequential(
    [
        Dense(units=6,activation=relu,name='l1'),
        Dense(units=6,activation=linear,name='l2')
    ],name = "Simple"
)

model_simple.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.01)
)

model_simple.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
13/13 [==============================] - 1s 2ms/step - loss: 2.0111
Epoch 2/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.6314
Epoch 3/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.3824
Epoch 4/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.1973
Epoch 5/1000
13/13 [==============================] - 0s 2ms/step - loss: 1.0417
Epoch 6/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.8899
Epoch 7/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.7635
Epoch 8/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.6569
Epoch 9/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5757
Epoch 10/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.5165
Epoch 11/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4697
Epoch 12/1000
13/13 [==============================] - 0s 2ms/step - loss: 0.4373
Epoch 13/1000
13/13 [====

In [63]:
model_simple.summary()

Model: "Simple"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 l1 (Dense)                  (None, 6)                 18        
                                                                 
 l2 (Dense)                  (None, 6)                 42        
                                                                 
Total params: 60 (480.00 Byte)
Trainable params: 60 (480.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [87]:
pre_train = model_simple.predict(X_train)
pre_train = tf.nn.softmax(pre_train).numpy()
yhat_train = np.argmax(pre_train, axis=1)
err_train = cat_err(y_train,yhat_train)

pre_cv = model_simple.predict(X_cv)
pre_cv = tf.nn.softmax(pre_cv).numpy()
yhat_cv = np.argmax(pre_cv, axis=1)
err_cv = cat_err(y_cv,yhat_cv)

print(f"Simple Model    training err {err_train:0.3f}, cv err {err_cv:0.3f}")
print(f"Complex Model   training err {train_err:0.3f}, cv err {cv_err:0.3f}")

10/10 [==============================] - 0s 3ms/step
Simple Model    training err 0.070, cv err 0.066
Complex Model   training err 0.013, cv err 0.103


In [83]:
# But simplize neural network is not the best way to fix high variance
# Let's try Regularization
# Reconstruct your complex model, but this time include regularization. Below, compose a three-layer model:

# Dense layer with 120 units, relu activation, kernel_regularizer=tf.keras.regularizers.l2(0.1)
# Dense layer with 40 units, relu activation, kernel_regularizer=tf.keras.regularizers.l2(0.1)
# Dense layer with 6 units and a linear activation. Compile using
# loss with SparseCategoricalCrossentropy, remember to use from_logits=True
# Adam optimizer with learning rate of 0.01.
tf.random.set_seed(1234)
model_regu = Sequential(
    [
        Dense(units=120,activation=relu,kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        Dense(units=40,activation=relu,kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        Dense(units=6,activation=linear)
    ],name = "Regularization"
)

model_regu.compile(
    loss=SparseCategoricalCrossentropy(from_logits=True),
    optimizer=Adam(learning_rate=0.01)
)

model_regu.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
13/13 [==============================] - 2s 8ms/step - loss: 4.5289
Epoch 2/1000
13/13 [==============================] - 0s 6ms/step - loss: 1.7001
Epoch 3/1000
13/13 [==============================] - 0s 6ms/step - loss: 1.3349
Epoch 4/1000
13/13 [==============================] - 0s 8ms/step - loss: 1.0749
Epoch 5/1000
13/13 [==============================] - 0s 10ms/step - loss: 0.9794
Epoch 6/1000
13/13 [==============================] - 0s 10ms/step - loss: 0.9644
Epoch 7/1000
13/13 [==============================] - 0s 11ms/step - loss: 0.9317
Epoch 8/1000
13/13 [==============================] - 0s 9ms/step - loss: 0.8748
Epoch 9/1000
13/13 [==============================] - 0s 15ms/step - loss: 0.7994
Epoch 10/1000
13/13 [==============================] - 0s 11ms/step - loss: 0.7584
Epoch 11/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.7475
Epoch 12/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.7477
Epoch 13/1000
13/13 

In [79]:
model_regu.summary()

Model: "Regularization"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 120)               360       
                                                                 
 dense_7 (Dense)             (None, 40)                4840      
                                                                 
 dense_8 (Dense)             (None, 6)                 246       
                                                                 
Total params: 5446 (42.55 KB)
Trainable params: 5446 (42.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [84]:
pre_train = model_regu.predict(X_train)
pre_train = tf.nn.softmax(pre_train).numpy()
yhat_train = np.argmax(pre_train, axis=1)

pre_cv = model_regu.predict(X_cv)
pre_cv = tf.nn.softmax(pre_cv).numpy()
yhat_cv = np.argmax(pre_cv, axis=1)

err_train_regu = cat_err(y_train,yhat_train)
err_cv_regu = cat_err(y_cv,yhat_cv)

print(f"Regularization Model    training err {err_train_regu:0.3f}, cv err {err_cv_regu:0.3f}")
print(f"Simple Model    training err {err_train:0.3f}, cv err {err_cv:0.3f}")
print(f"Complex Model   training err {train_err:0.3f}, cv err {cv_err:0.3f}")

10/10 [==============================] - 0s 4ms/step
Regularization Model    training err 0.068, cv err 0.066
Simple Model    training err 0.072, cv err 0.078
Complex Model   training err 0.007, cv err 0.109


In [89]:
# Iterate to find optimal regularization value
tf.random.set_seed(1234)
lambdas = [0.0, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3]
err_train = np.zeros(len(lambdas))
err_cv = np.zeros(len(lambdas))

for i in range(len(lambdas)):
  models_i = Sequential(
      [
          Dense(units=120,activation=relu,kernel_regularizer=tf.keras.regularizers.l2(lambdas[i])),
          Dense(units=40,activation=relu,kernel_regularizer=tf.keras.regularizers.l2(lambdas[i])),
          Dense(units=6,activation=linear)
      ],name = "Regularization"
  )

  models_i.compile(
      loss=SparseCategoricalCrossentropy(from_logits=True),
      optimizer=Adam(learning_rate=0.01)
  )

  models_i.fit(X_train, y_train, epochs=1000)

  pre_train_i = models_i.predict(X_train)
  pre_train_i = tf.nn.softmax(pre_train_i).numpy()
  yhat_train_i = np.argmax(pre_train_i, axis=1)
  pre_cv_i = models_i.predict(X_cv)
  pre_cv_i = tf.nn.softmax(pre_cv_i).numpy()
  yhat_cv_i = np.argmax(pre_cv_i, axis=1)

  err_train[i] = cat_err(y_train,yhat_train_i)
  err_cv[i] = cat_err(y_cv,yhat_cv_i)

print(err_train)
print(err_cv)


Streaming output truncated to the last 5000 lines.
Epoch 505/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.3652
Epoch 506/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.3784
Epoch 507/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.3515
Epoch 508/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.3368
Epoch 509/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.3501
Epoch 510/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.3469
Epoch 511/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.3493
Epoch 512/1000
13/13 [==============================] - 0s 3ms/step - loss: 0.3606
Epoch 513/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.3514
Epoch 514/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.3481
Epoch 515/1000
13/13 [==============================] - 0s 4ms/step - loss: 0.3509
Epoch 516/1000
13/13 [==============

In [90]:
def output_to_err(model,X,y):
  err = 0
  pre = model.predict(X)
  pre = tf.nn.softmax(pre).numpy()
  yhat = np.argmax(pre, axis=1)
  err = cat_err(y,yhat)
  return err

In [91]:
err_test_complex = output_to_err(model,X_test,y_test)
err_test_simple = output_to_err(model_simple,X_test,y_test)
err_test_regu = output_to_err(model_regu,X_test,y_test)

print(f"Complex Model           test err {err_test_complex:0.3f}")
print(f"Simple Model            test err {err_test_simple:0.3f}")
print(f"Regularization Model    test err {err_test_regu:0.3f}")


3/3 [==============================] - 0s 3ms/step
Complex Model           test err 0.138
Simple Model            test err 0.163
Regularization Model    test err 0.138
